### Degree centrality Removal node

In [1]:
import networkx as nx
import pandas as pd

G = nx.read_graphml('london.graph.xml')
for node in G.nodes():
    G.nodes[node]['coords'] = eval(G.nodes[node]['coords'])

deg_london = nx.degree_centrality(G)
nx.set_node_attributes(G, dict(deg_london), 'degree')

df = pd.DataFrame(index=G.nodes())
df['station_name'] = pd.Series(nx.get_node_attributes(G, 'station_name'))
df['degree'] = pd.Series(nx.get_node_attributes(G, 'degree'))
df_sorted = df.sort_values(["degree"], ascending=False)

top_3_nodes = df_sorted.head(3).index
G.remove_nodes_from(top_3_nodes)

def Weighted_Global_Efficiency(graph):
    if len(graph) <= 1:
        return 0
    N = len(graph)
    weighted_inverse_path_length_sum = 0.0
    total_flow_sum = 0.0
    for j in graph.nodes():
        for k in graph.nodes():
            if j != k:
                try:
                    shortest_path = nx.shortest_path(graph, source=j, target=k)
                    path_length = sum(graph[shortest_path[i]][shortest_path[i + 1]]['length']
                    for i in range(len(shortest_path) - 1))
                    path_flow = sum(graph[shortest_path[i]][shortest_path[i + 1]]['flows'] for i in range(len(shortest_path) - 1))
                    weighted_inverse_path_length_sum += (path_flow * (1 / path_length))
                    total_flow_sum += path_flow
                except nx.NetworkXNoPath:
                    continue
    if total_flow_sum == 0:
        return 0
    mean_weighted_inverse_path_length = weighted_inverse_path_length_sum / total_flow_sum
    return mean_weighted_inverse_path_length




def modified_weighted_lcc(graph):
    if len(graph) == 0:
        return 0

    component_flows = {}
    
    for component in nx.connected_components(graph):
        weighted_flow = 0  # Use this to accumulate the weighted flow
        component_tuple = tuple(component)
        
        # Iterate through each node in the component
        for node in component:
            # Iterate through each neighbor of the node
            for neighbor in graph.neighbors(node):
                # Check if the neighbor is in the same component
                if neighbor in component:
                    edge_data = graph.get_edge_data(node, neighbor)
                    # If there is edge data, calculate the weighted flow
                    if edge_data:
                        flow = edge_data.get('flows', 0)
                        length = edge_data.get('length', 1)  # Default length to 1 if not present
                        weighted_flow += flow * length  # Weighted by the length of the edge
                        
        component_flows[component_tuple] = weighted_flow

    largest_flow_component = max(component_flows.values()) if component_flows else 0
    total_weighted_flows = sum(component_flows.values())
    return largest_flow_component / total_weighted_flows if total_weighted_flows else 0



edge_weighted_mean_inv_shortest_path = Weighted_Global_Efficiency(G)
weighted_lcc = modified_weighted_lcc(G)

print(f"Modified-Weighted-Efficiency: {edge_weighted_mean_inv_shortest_path}")
print(f"Modified-Weighted-NLCC: {weighted_lcc}")


Modified-Weighted-Efficiency: 4.887152920206759e-05
Modified-Weighted-NLCC: 0.9664325266623214


##### Which Stations impact more

In [36]:
import networkx as nx
import pandas as pd

G = nx.read_graphml('london.graph.xml')
for node in G.nodes():
    G.nodes[node]['coords'] = eval(G.nodes[node]['coords'])

deg_centrality = nx.degree_centrality(G)
nx.set_node_attributes(G, dict(deg_centrality), 'degree')

df = pd.DataFrame(index=G.nodes())
df['station_name'] = pd.Series(nx.get_node_attributes(G, 'station_name'))
df['degree'] = pd.Series(nx.get_node_attributes(G, 'degree'))
df_sorted = df.sort_values(["degree"], ascending=False)

top_3_nodes = df_sorted.head(3).index.tolist()

original_graph = G.copy()
impact_dict = {}

for node in top_3_nodes:
    G = original_graph.copy()
    G.remove_node(node)
    
    mwmi = Weighted_Global_Efficiency(G)
    
    mwlc = modified_weighted_lcc(G)
    impact_dict[node] = {
        'Modified-Weighted-NISPL': mwmi,
        'Modified-Weighted-NLCC': mwlc
    }

for node, impacts in impact_dict.items():
    print(f"Node: {node}, Impacts: {impacts}")


Node: Stratford, Impacts: {'Modified-Weighted-NISPL': 5.26664798547187e-05, 'Modified-Weighted-NLCC': 0.9706874259407939}
Node: Bank and Monument, Impacts: {'Modified-Weighted-NISPL': 4.974414973479428e-05, 'Modified-Weighted-NLCC': 1.0}
Node: King's Cross St. Pancras, Impacts: {'Modified-Weighted-NISPL': 4.6814694718893994e-05, 'Modified-Weighted-NLCC': 1.0}


In [3]:
import networkx as nx
import pandas as pd
import copy

G = nx.read_graphml('london.graph.xml')
for node in G.nodes():
    G.nodes[node]['coords'] = eval(G.nodes[node]['coords'])
deg_london = nx.degree_centrality(G)
nx.set_node_attributes(G, dict(deg_london), 'degree')
df = pd.DataFrame(index=G.nodes())
df['station_name'] = pd.Series(nx.get_node_attributes(G, 'station_name'))
df['degree'] = pd.Series(nx.get_node_attributes(G, 'degree'))
df_sorted = df.sort_values("degree", ascending=False)


N = 3
impact_results = []

for node in df_sorted.head(N).index:
    H = copy.deepcopy(G) 
    H.remove_node(node)

    edge_weighted_mean_inv_shortest_path = Weighted_Global_Efficiency(H)
    weighted_lcc = modified_weighted_lcc(H)

    impact_results.append({
        'station_name': df.loc[node, 'station_name'],
        'node': node,
        'Modified-Weighted-NISPL': edge_weighted_mean_inv_shortest_path,
        'Modified-Weighted-NLCC': weighted_lcc
    })

impact_df = pd.DataFrame(impact_results)
print(impact_df)


   station_name                      node  Modified-Weighted-NISPL  \
0           NaN                 Stratford                 0.000053   
1           NaN         Bank and Monument                 0.000050   
2           NaN  King's Cross St. Pancras                 0.000047   

   Modified-Weighted-NLCC  
0                0.970687  
1                1.000000  
2                1.000000  


### Betweeness centrality Removal node

In [5]:
import networkx as nx
import pandas as pd

G = nx.read_graphml('london.graph.xml')
for node in G.nodes():
    G.nodes[node]['coords'] = eval(G.nodes[node]['coords'])

bet_london =nx.betweenness_centrality(G)
nx.set_node_attributes(G, dict(bet_london), 'degree')

df = pd.DataFrame(index=G.nodes())
df['station_name'] = pd.Series(nx.get_node_attributes(G, 'station_name'))
df['degree'] = pd.Series(nx.get_node_attributes(G, 'degree'))
df_sorted = df.sort_values(["degree"], ascending=False)

top_3_nodes = df_sorted.head(3).index
G.remove_nodes_from(top_3_nodes)

def Weighted_Global_Efficiency(graph):
    if len(graph) <= 1:
        return 0
    N = len(graph)
    weighted_inverse_path_length_sum = 0.0
    total_flow_sum = 0.0
    for j in graph.nodes():
        for k in graph.nodes():
            if j != k:
                try:
                    shortest_path = nx.shortest_path(graph, source=j, target=k)
                    path_length = sum(graph[shortest_path[i]][shortest_path[i + 1]]['length']
                    for i in range(len(shortest_path) - 1))
                    path_flow = sum(graph[shortest_path[i]][shortest_path[i + 1]]['flows'] for i in range(len(shortest_path) - 1))
                    weighted_inverse_path_length_sum += (path_flow * (1 / path_length))
                    total_flow_sum += path_flow
                except nx.NetworkXNoPath:
                    continue
    if total_flow_sum == 0:
        return 0
    mean_weighted_inverse_path_length = weighted_inverse_path_length_sum / total_flow_sum
    return mean_weighted_inverse_path_length

def modified_weighted_lcc(graph):
    if len(graph) == 0:
        return 0

    component_flows = {}
    
    for component in nx.connected_components(graph):
        total_flow = 0
        component_tuple = tuple(component)
        for node in component:
            for neighbor in graph.neighbors(node):
                if neighbor in component:
                    edge_data = graph.get_edge_data(node, neighbor)
                    if edge_data:
                        total_flow += edge_data.get('flows', 0)
        component_flows[component_tuple] = total_flow

    largest_flow_component = max(component_flows.values()) if component_flows else 0
    total_flows = sum(component_flows.values())
    return largest_flow_component / total_flows if total_flows else 0

edge_weighted_mean_inv_shortest_path = Weighted_Global_Efficiency(G)
weighted_lcc = modified_weighted_lcc(G)

print(f"Modified-Weighted-NISPL: {edge_weighted_mean_inv_shortest_path}")
print(f"Modified-Weighted-NLCC: {weighted_lcc}")


Modified-Weighted-NISPL: 5.0826074219808845e-05
Modified-Weighted-NLCC: 0.9745390285039769


##### Which Stations impact more

In [6]:
import networkx as nx
import pandas as pd
import copy

G = nx.read_graphml('london.graph.xml')
for node in G.nodes():
    G.nodes[node]['coords'] = eval(G.nodes[node]['coords'])
deg_london =nx.degree_centrality(G)
nx.set_node_attributes(G, dict(deg_london), 'degree')
df = pd.DataFrame(index=G.nodes())
df['station_name'] = pd.Series(nx.get_node_attributes(G, 'station_name'))
df['degree'] = pd.Series(nx.get_node_attributes(G, 'degree'))
df_sorted = df.sort_values("degree", ascending=False)


N = 3
impact_results = []

for node in df_sorted.head(N).index:
    H = copy.deepcopy(G) 
    H.remove_node(node)

    edge_weighted_mean_inv_shortest_path = Weighted_Global_Efficiency(H)
    weighted_lcc = modified_weighted_lcc(H)

    impact_results.append({
        'station_name': df.loc[node, 'station_name'],
        'node': node,
        'Modified-Weighted-NISPL': edge_weighted_mean_inv_shortest_path,
        'Modified-Weighted-NLCC': weighted_lcc
    })

impact_df = pd.DataFrame(impact_results)
print(impact_df)


   station_name                      node  Modified-Weighted-NISPL  \
0           NaN                 Stratford                 0.000053   
1           NaN         Bank and Monument                 0.000050   
2           NaN  King's Cross St. Pancras                 0.000047   

   Modified-Weighted-NLCC  
0                0.976435  
1                1.000000  
2                1.000000  
